In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install pymupdf4llm

!pip install sentence-transformers
!pip install faiss-gpu
!pip install langchain-teddynote

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

from google.colab import drive
drive.mount("/content/drive")
import pandas as pd

Mounted at /content/drive


In [ ]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from langchain_community.document_loaders import PDFMinerLoader

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever
# pdf to MD
import pymupdf4llm
import re

In [ ]:
!unzip -qq '/content/drive/MyDrive/Dacon/재정정보 AI 검색 알고리즘 경진대회/data/open.zip'

In [ ]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=50):
    """PDF to Markdown"""
    # PDF 파일 열기
    md_text = pymupdf4llm.to_markdown(file_path)
    headers = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    # 마크다운 헤더를 기준으로 분할
    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers, strip_headers=False)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size= chunk_size, chunk_overlap= chunk_overlap
    )

    chunks = text_splitter.split_documents(md_chunks)
    return chunks

def create_vector_db(chunks, model_path="BAAI/bge-m3"):
    """FAISS DB 생성, 임베딩 모델 설정"""

    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    # FAISS DB 생성
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)

        # Retriever 생성
        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
        faiss_retriever = db.as_retriever()

        retriever = EnsembleRetriever(
        retrievers=[kiwi_bm25_retriever, faiss_retriever],
        weights=[0.6, 0.4], # bm25_w, faiss_w
        search_type="mmr",
)
        # 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

In [ ]:
base_directory = '/content'
df = pd.read_csv('/content/test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

Processing PDFs:  78%|███████▊  | 7/9 [01:30<00:24, 12.33s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [01:43<00:12, 12.57s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [01:58<00:00, 13.15s/it]


In [ ]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model_id = "rtzr/ko-gemma-2-9b-it"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text= False,
        max_new_tokens=512,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf
llm = setup_llm_pipeline()

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

...

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context_set = set()
    for doc in docs:
        context_set.add(doc.page_content)
    context = '\n'.join(context_set)
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    제공된 정보를 바탕으로 주어진 질문에 대하여 답해주세요.
    정보: {context}

    지켜야할 규칙은 다음과 같습니다.
    1. 제공된 context를 바탕으로 질문에 대한 답변을 해주세요.
    2. 불필요한 설명은 지양하고 답변에 해당되는 내용만 출력해주세요.
    3. 답변은 간결하게 몇 문장으로만 하고 중간에 끊기지 않도록 해주세요.
    4. 같은 내용 또는 같은 문장을 반복하지 말아주세요.
    5. 답변은 존댓말로 해주세요.
    6. 답변을 정확하게 제공하면 팁으로 1만 달러를 제공해드리겠습니다.

    질문은 다음과 같습니다.
    질문: {question}

    답변:
    """

    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:06<10:10,  6.29s/it]

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2,300,000만원입니다. 


Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 2/98 [00:23<20:32, 12.84s/it]

...

Answer: 국가결산보고서는 '우발사항 및 약정사항'으로, 지방자치단체 회계기준은 '우발상황', '우발손실', '우발이익'으로 각각 정의하고 있습니다. 따라서 중앙정부와 지방정부의 우발부채에 대한 인식과 회계처리가 상이합니다. 




Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Answering Questions:  99%|█████████▉| 97/98 [19:44<00:14, 14.58s/it]

Answer: 우발부채는 미래 발생이 불확실한 잠재적 의무 또는 부채로, 과거의 거래나 사건으로 발생했지만, 현재의무로 인식되지 않거나 금액을 신뢰성 있게 추정하기 어려운 경우를 말합니다.  우발부채는 재정위험을 발생시키고, 국제기준 재정통계 산출에 따라 발생주의 회계기준이 적용되면서 그 관리 중요성이 더욱 부각되었습니다. 




Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


Answering Questions: 100%|██████████| 98/98 [20:11<00:00, 12.36s/it]

Answer: 보증은 특정 조건이 충족될 경우, 보증인이 보증받는 대상의 의무를 대체하거나 이행하는 것을 약속하는 것을 말합니다. 파생금융상품 형태의 보증과 표준화 보증은 재정상태표에 부채로 기록됩니다. 표준화 보증은 통상 아주 적은 금액에 대해 획일적 조건으로 대규모로 발행되는 보증으로, 수출신용 보증, 환보증, 다양한 종류의 보험, 농민융자, 모기지론, 학자금융자, 중소기업융자 등이 있습니다. 표준화 보증은 주로 유사한 특징을 지니고 위험 분산을 목적으로 발행됩니다. 






In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("/content/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("해당 정보를 찾을 수 없습니다.")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("/content/drive/MyDrive/Dacon/재정정보 AI 검색 알고리즘 경진대회/result/ko-gemma-2-9b.csv", encoding='UTF-8-sig', index=False)